In [2]:
import mysql.connector
# library for data analysis
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 

import os 

In [3]:
# Membuat koneksi ke database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='tourista_db'
)

In [4]:
# Mengeksekusi query untuk mengambil data dari tabel

query = "SELECT * FROM destination"
destination = pd.read_sql_query(query, conn)

query = "SELECT * FROM review_wisata"
ratings = pd.read_sql_query(query, conn)

query = "SELECT * FROM user_profile"
users = pd.read_sql_query(query, conn)

C:\Users\dewan\AppData\Local\Temp\ipykernel_1116\707316895.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  destination = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_1116\707316895.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_1116\707316895.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = pd.read_sql_query(query, conn)


In [19]:
# new variable for content based filtering 
place_id = destination['id'].unique().tolist()
place_name = destination['name_wisata'].unique().tolist()
place_category = destination['category'].tolist()

In [20]:
# dictionary for place_recommend
place_recommend = pd.DataFrame({
    'place_id' : place_id,
    'place_name' : place_name,
    'place_category' : place_category
})
place_recommend

,place_id,place_name,place_category
0,1,Monumen Nasional,Budaya
1,2,Kota Tua,Budaya
2,3,Dunia Fantasi,Taman Hiburan
3,4,Taman Mini Indonesia Indah (TMII),Taman Hiburan
4,5,Atlantis Water Adventure,Taman Hiburan
...,...,...,...
432,433,Museum Mpu Tantular,Budaya
433,434,Taman Bungkul,Taman Hiburan
434,435,Taman Air Mancur Menari Kenjeran,Taman Hiburan
435,436,Taman Flora Bratang Surabaya,Taman Hiburan


In [21]:
# data frame for content based filtering 
data_content = place_recommend
data_content

,place_id,place_name,place_category
0,1,Monumen Nasional,Budaya
1,2,Kota Tua,Budaya
2,3,Dunia Fantasi,Taman Hiburan
3,4,Taman Mini Indonesia Indah (TMII),Taman Hiburan
4,5,Atlantis Water Adventure,Taman Hiburan
...,...,...,...
432,433,Museum Mpu Tantular,Budaya
433,434,Taman Bungkul,Taman Hiburan
434,435,Taman Air Mancur Menari Kenjeran,Taman Hiburan
435,436,Taman Flora Bratang Surabaya,Taman Hiburan


In [22]:
tfidf= TfidfVectorizer()
tfidf.fit(data_content['place_name'])

TfidfVectorizer()

In [23]:
# change data into matrix 
tfidf_matrix = tfidf.fit_transform(data_content['place_name'])
tfidf_matrix.shape

(437, 712)

In [24]:
# change vector TF-IDF into matrix 
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
# calculate the cosine similairty between the place name and each entry in 
cosine_similarity = cosine_similarity(tfidf_matrix)
cosine_similarity

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.07709925,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.07709925, 1.        ,
        0.13442662],
       [0.        , 0.        , 0.        , ..., 0.        , 0.13442662,
        1.        ]])

In [26]:
df_cosine_similarity = pd.DataFrame(cosine_similarity, index = data_content['place_name'], columns = data_content['place_name'])
print('Shape : ', df_cosine_similarity.shape)

Shape :  (437, 437)


In [29]:
# recommendation function for content based filtering 

def content_rec(place_name, similarity_data = df_cosine_similarity, items = data_content[['place_name', 'place_category']], k = 5) :
  index = similarity_data.loc[:, place_name].to_numpy().argpartition(range(-1, -k, -1))

  closest = similarity_data.columns[index[-1:-(k+2): -1]]
  closest = closest.drop(place_name, errors = 'ignore')

  return pd.DataFrame(closest).merge(items).head(k)

In [30]:
# test the function 
content_rec('Alun-Alun Kota Bandung')

,place_name,place_category
0,Alun Alun Selatan Yogyakarta,Taman Hiburan
1,Alun-alun Utara Keraton Yogyakarta,Budaya
2,Taman Balai Kota Bandung,Taman Hiburan
3,Kota Mini,Taman Hiburan
4,Kota Tua,Budaya
